# Import modules

In [33]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Utils

In [27]:
def display_formatted_time(elapsed_time, msg=""):
    minutes, seconds = map(int, divmod(elapsed_time, 60));
    print("Elapsed time - {0}: {1}min {2}s".format(msg, minutes, seconds))

# Define the network

In [7]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  # 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [23]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # Negative Log Likelihood Loss
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)
        ))

# Settings

In [26]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

torch.manual_seed(1)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


batch_size = 64
test_batch_size = 1000
epochs = 3
log_interval = 10
lr = 0.01
momentum = 0.5

Device:  cpu


# Load data

In [15]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True,
                                transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size,
    shuffle=True,
    **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=test_batch_size,
    shuffle=True,
    **kwargs)

# Train / Test

In [32]:
since = time.time()

model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                                         lr=lr,
                                         momentum=momentum
                      )

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

display_formatted_time(time.time() - since)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.281469
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.316734
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285961
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.267647
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.255398
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.224318
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.221539
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.218056
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.136967
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.956032
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.094250
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.811190
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.790050
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.679816
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.431165
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.538860
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.362196
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.288106
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.207235
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.070003
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.319803
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.376538
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.408045
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.113500
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.277269
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.178950
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.276028
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.361176
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.392511
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.324430
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.500845
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.711447
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.252570
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.333636
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.382427
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.221759
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.282758
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.193320
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.498053
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.310894
